In [1]:
import pandas as pd
from pymongo import MongoClient
import random

from lib.mongo_aggregation_verbs import *

mongo_client = MongoClient('this-mongo.cc', 27016)
database_reference = mongo_client.twitter

In [2]:
database_reference.collection_names()

collection_reference = database_reference.instructor_test_group

test_group = database_reference.instructor_test_group

In [3]:
match_empty_url_arrays = { MATCH : { "entities.urls" : []}}

list(test_group.aggregate(
    [
        match_empty_url_arrays,
        { COUNT : "text"}
    ]
))

[{'text': 11121}]

In [4]:
job_hashtags = ['job', 'jobs', 'hiring', 'careerarc']
location_hashtags = ['california', 'losangeles', 'la', 'santamonica', 'glendale', 'paloalto']
match_not_in_bad = { MATCH : { "text" : { "$in" : job_hashtags + location_hashtags}}}
project_to_text_keep_id = { PROJECT : { "text" : "$entities.hashtags.text" } }
project_to_id = { PROJECT : { "_id" : 1 } }

bad_ids = list(test_group.aggregate(
    [
        match_non_empty_hashtag_arrays,
        project_to_text_keep_id,
        unwind_text,
        project_to_lower,
        match_not_in_bad,
        project_to_id
    ]
))
bad_ids[:10], len(bad_ids)

([{'_id': ObjectId('5a73683636dd5f00016c7fad')},
  {'_id': ObjectId('5a73683636dd5f00016c7fad')},
  {'_id': ObjectId('5a73683636dd5f00016c7fad')},
  {'_id': ObjectId('5a73683636dd5f00016c7fad')},
  {'_id': ObjectId('5a73683636dd5f00016c7fad')},
  {'_id': ObjectId('5a6da1bd36dd5f0001690696')},
  {'_id': ObjectId('5a6da1bd36dd5f0001690696')},
  {'_id': ObjectId('5a6da1bd36dd5f0001690696')},
  {'_id': ObjectId('5a6df39136dd5f0001691533')},
  {'_id': ObjectId('5a6df39136dd5f0001691533')}],
 1835)

In [5]:
bad_ids = [bad_id['_id'] for bad_id in bad_ids]
bad_ids[:10]

[ObjectId('5a73683636dd5f00016c7fad'),
 ObjectId('5a73683636dd5f00016c7fad'),
 ObjectId('5a73683636dd5f00016c7fad'),
 ObjectId('5a73683636dd5f00016c7fad'),
 ObjectId('5a73683636dd5f00016c7fad'),
 ObjectId('5a6da1bd36dd5f0001690696'),
 ObjectId('5a6da1bd36dd5f0001690696'),
 ObjectId('5a6da1bd36dd5f0001690696'),
 ObjectId('5a6df39136dd5f0001691533'),
 ObjectId('5a6df39136dd5f0001691533')]

In [6]:
not_in_bad_ids_and_no_url = { "_id" : {"$nin" : bad_ids}, "entities.urls" : []}
just_the_text = {"text" : 1, "_id" : 0}

In [7]:
test_group.find_one(
    not_in_bad_ids_and_no_url,
    just_the_text
)

{'text': "@SincerelyLegit Lol why at night? If it's boring I'll fall asleep"}

In [8]:
cur  = test_group.find(
    not_in_bad_ids_and_no_url,
    just_the_text
)

tweets = list(cur)
tweet_text = pd.DataFrame(tweets)

In [9]:
len(tweet_text)

11102

In [10]:
tweet_text.head()

,text
0,@SincerelyLegit Lol why at night? If it's bori...
1,@godtributes Lol
2,@KingBeyonceStan I need to binge on both now b...
3,@_QUEENSharnay Naw I’d i can’t put this skip o...
4,@sannicolaso I have things to do at home but y...


In [11]:
tweet_text.text = tweet_text.text.str.replace('http\S+|www.\S+', '', case=False)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')

tfidf.fit(tweet_text.text)
word_occurence = tfidf.transform(tweet_text.text).todense()

MemoryError: 

In [ ]:
words = tfidf.get_feature_names()
word_sample = random.sample(words, 20)
word_occurence_m = pd.DataFrame(word_occurence, columns=words)
word_occurence_m[word_sample].head()

In [15]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_topics=10)
lda.fit(word_occurence)

lda_df = pd.DataFrame(lda.components_, columns=words).T

NameError: name 'word_occurence' is not defined

In [13]:
def filter_topic(lda_df, index, threshold):
    return (lda_df[lda_df[index] > threshold][index]
            .sort_values(ascending=False))

In [14]:
filter_topic(lda_df, 0, 15)

NameError: name 'lda_df' is not defined

In [ ]:
filter_topic(lda_df, 1, 15)

In [ ]:
filter_topic(lda_df, 2, 15)

In [ ]:
filter_topic(lda_df, 3, 10)

In [ ]:
filter_topic(lda_df, 4, 20)

In [ ]:
filter_topic(lda_df, 5, 25)

In [ ]:
filter_topic(lda_df, 6, 20)

In [ ]:
filter_topic(lda_df, 7, 15)

In [ ]:
filter_topic(lda_df, 8, 25)

In [ ]:
filter_topic(lda_df, 9, 10)